In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None
import numpy as np
 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set('talk')
sns.set_style('white')

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm import tqdm
tqdm.pandas()

In [2]:
train = pd.read_csv('../../data/train.csv')
test = pd.read_csv('../../data/test.csv')
users = pd.read_csv('../../data/users.csv')

In [3]:
train = pd.merge(train, users, on=['user_id'], how='left')
test = pd.merge(test, users, on=['user_id'], how='left')

In [4]:
train['domain'] = train['domain'].replace(r'@(\w+).com', r'\1', regex=True)
test['domain'] = test['domain'].replace(r'@(\w+).com', r'\1', regex=True)

In [5]:
encoder = LabelEncoder()

train['domain'] = encoder.fit_transform(train['domain'])
test['domain'] = encoder.transform(test['domain'])

In [6]:
left = train[train['country_code'] != 1]
x_prime = len(left)
n = int((0.3*x_prime)/(0.7))
sample = train[train['country_code'] == 1].sample(n=n, random_state=42)
train = pd.concat([sample, left], axis=0)

In [7]:
min_sub = test['subject_line_length'].min()
max_sub = test['subject_line_length'].max()
train = train[(train['subject_line_length'] <= max_sub) & (train['subject_line_length'] >= min_sub)]

In [8]:
probe_positive_rate = 0.147
number_positive_required = int(0.147 * len(train))
left = train[train['open_flag'] == 0]
sample = train[train['open_flag'] == 1].sample(n=number_positive_required, random_state=42)
train = pd.concat([sample, left], axis=0)
train['open_flag'].mean()

0.15001908639776054

In [9]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)